In [1]:
#1
def deco(func):
  def inner():
    print('running inner()')
  return inner


In [2]:
@deco
def target():
  print("running target()")
target()

running inner()


In [3]:
target

<function __main__.deco.<locals>.inner()>

In [4]:
#1-1

def logger(func):
  def wrapper(*args,**kwargs):
    print(f"[LOG] {func.__name__} called with ars={args}. kwargs={kwargs}")
    result=func(*args, **wargs)
    print(f"[LOG] {func.__name__}returned {result}")
    return result
  return wrapper

@logger
def add(a,b):
  return a+b

In [5]:
#2
registry=[]
def register(func):
  print('running regsiter(%s)' %func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

@register
def f3():
  print('running f3()')

def main():
  print('running main()')
  print('register=>',register)
  f1()
  f2()
  f3()

if __name__=='__main__':
  main()



running regsiter(<function f1 at 0x797d28db5b20>)
running regsiter(<function f2 at 0x797d28db5800>)
running regsiter(<function f3 at 0x797d28db5440>)
running main()
register=> <function register at 0x797d28db4860>
running f1()
running f2()
running f3()


In [6]:
#3
promos=[]

def promotion(promo_func):
  """promotion() decorater는 promo_func를 promos리스트에 추가한 후 그대로 반환"""
  promos.append(promo_func)
  return promo_func

@promotion
def fidelity_promo(order):
  """충성도 점수가 1000점 이상인 고객에게 전체 5%할인 적용"""
  return order.total() *0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
  """20개 이상의 동일 상품을구입하면 10% 할인 적용"""
  discount=0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total()*0.1
    return discount

@promotion
def large_order_promo(order):
  """10종류 이상의 ㅏㅇ품을 구입하면 10%할인 적용"""
  distinct_item={item.product for item in order.cart}
  if len(distinct_item) >=10:
    return order.total()*0.07
  return 0

def best_promo(order):
  """최대로할인받을 금액을 반환한다."""
  return max(promo(order) for promo in promos)

In [7]:
#4
def f1(a):
  print(a)
  print(b)
f1(3)

3


NameError: name 'b' is not defined

In [8]:
#5
b=6
def f2(a):
  print(a)
  print(b)
  b=9
f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [9]:
#6

b=6
def f3(a):
  global b
  print(a)
  print(b)
  b=9
f3(3)

3
6


In [10]:
f3(3)

3
9


In [11]:
b=30
f3(3)

3
30


In [12]:
#7
class Averager():
  def __init__(self):
    self.series=[]

  def __call__(self, new_value):
    self.series.append(new_value)
    total=sum(self.series)
    return total/len(self.series)

In [13]:
avg=Averager()
avg(10)

10.0

In [14]:
avg(11)

10.5

In [15]:
avg(12)

11.0

In [16]:
#8
def make_averager():
  series=[]

  def averager(new_value):
    series.append(new_value)
    total =sum(series)
    return total/len(series)

  return averager

In [17]:
avg=make_averager()
avg(10)

10.0

In [18]:
avg(11)

10.5

In [19]:
avg(12)

11.0

In [20]:
#9
avg.__code__.co_varnames

('new_value', 'total')

In [21]:
avg.__code__.co_freevars

('series',)

In [22]:
avg.__closure__

(<cell at 0x797d28e283d0: list object at 0x797d28ecedc0>,)

In [23]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [24]:
#10
def make_averager():
  count=0
  total=0

  def averager(new_value):
    count +=1
    total +=new_value
    return total/count

  return averager

In [25]:
avg=make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [26]:
#11
def make_averager():
  count=0
  total=0

  def averager(new_value):
    nonlocal count, total
    count +=1
    total += new_value
    return total/count
  return averager

avg=make_averager()
avg(10)

10.0

In [27]:
#12
import time

def clock(func):
  def clocked(*args):
    t0=time.perf_counter()
    result=func(*args)
    elapsed=time.perf_counter()-t0
    name=func.__name__
    arg_str =','.join(repr(arg) for arg in args)
    print('[%0.8fs]%s(%s)-> %r'%(elapsed,name,arg_str,result))
    return result
  return clocked

In [28]:
#13

@clock
def snooze(seconds):
  time.sleep(seconds)

@clock

def factorial(n):
  return 1 if n<2 else n*factorial(n-1)

if __name__=='__main__':
  print('*'*40,'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6!=',factorial(6))

**************************************** Calling snooze(.123)
[0.12315284s]snooze(0.123)-> None
**************************************** Calling factorial(6)
[0.00000055s]factorial(1)-> 1
[0.00001285s]factorial(2)-> 2
[0.00001876s]factorial(3)-> 6
[0.00002396s]factorial(4)-> 24
[0.00002958s]factorial(5)-> 120
[0.00003718s]factorial(6)-> 720
6!= 720


In [30]:
factorial.__name__

'clocked'

In [29]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        arg_lst = []
        name = func.__name__
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.9fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


In [31]:
#14
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n <2 else n*factorial(n-1)

if __name__=='__main__':
  print('*'*40, 'Caolling snooze(.123)')
  snooze(.123)
  print('*'*40,'Calling factorial(6)')
  print('6!=',factorial(6))

**************************************** Caolling snooze(.123)
[0.123100281s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.000000477s] factorial(1) -> 1
[0.000013590s] factorial(2) -> 2
[0.000020742s] factorial(3) -> 6
[0.000026464s] factorial(4) -> 24
[0.000032425s] factorial(5) -> 120
[0.000040531s] factorial(6) -> 720
6!= 720


In [ ]:
factorial.__name__

'factorial'